*   실행환경: 구글 코랩
*   작성일: 230930
*   작성자: 이혜원



### 라이브러리 설치, 불러오기

In [1]:
!pip install transformers
# !pip install konlpy
# !pip install mecab-python
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.3 MB/s eta 0:00:00


In [13]:
#라이브러리 불러오기
import numpy as np
import pandas as pd
import re

import random
import time
import datetime
from tqdm import tqdm

import csv
import os

import tensorflow as tf
import torch

# BERT 사용을 위함
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# for padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리 및 평가 지표
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, hamming_loss

from torch.utils.data import Dataset

from sklearn.preprocessing import MultiLabelBinarizer

ModuleNotFoundError: ignored

### 데이터 불러오기

#### 장르 스테디셀러, 가사 요약 데이터 불러오기

In [11]:
steady_sum_song_df = pd.read_excel('가사_스테디셀러_요약.xlsx')

NameError: ignored

#### 장르 스테디셀러 데이터 불러오기

In [12]:
steady_song_df = pd.read_csv('가사_스테디셀러.csv')

NameError: ignored

#### 차트파인더 데이터 불러오기

In [ ]:
song_df = pd.read_csv('가사.csv')
rank_df = pd.read_csv('순위.csv')
info_df = pd.read_csv('앨범정보.csv')

In [ ]:
print('song_df.shape:', song_df.shape)
print('rank_df.shape:', rank_df.shape)
print('info_df.shape:', info_df.shape)

### 사용자 정의 함수 만들기

함수




*   preprocess(sentences) : 가사 전처리
*   tokenize(sentences) : 형태소 분석기 mecab을 사용한 토큰화 작업 (다소 시간 소요)
*   find_rank_song(song_id) : song_id로 rank_df 행 찾기
*   find_rank_album(album_id) : album_id로 rank_df 행 찾기
*   find_info_id(album_id) : album_id로 info_df 행 찾기
*   find_genre_song(song_id) : song_id를 넣으면 genre를 반환
*   find_genres_songs(song_id_list) : song_id_list를 넣으면 genre_list를 반환
*   make_unique_lst(df_column) : unique한 list 구하기
*   make_word_to_idx(unique_lst) : 리턴 -> dict {"A" :1, "B" :2, ... }  형식
*   ⭐make_idx_list(column_lst) : 2차원 리스트의 단어를 숫자와 매핑


In [4]:
# 가사 전처리
def preprocess(sentences):
    lst = []
    for sentence in sentences:
        # \n과 \t를 제거한다
        sentence = re.sub('\\t', '', sentence)
        sentence = re.sub('\\n', '', sentence)
        # 한국어, 공백 빼고 제거하기
        sentence = re.sub('[^ㄱ-ㅎ가-힣\s]+', '', sentence)
        # 문장 양옆의 띄어쓰기를 지운다
        sentence = sentence.strip()

        lst.append(sentence)
    return lst

In [5]:
# 형태소 분석기 mecab을 사용한 토큰화 작업 (다소 시간 소요)
def tokenize(sentences):
    mecab = Mecab()
    tokenized_data = []
    for sentence in sentences:
        temp_X = mecab.morphs(sentence)
        temp_X = [word for word in temp_X]
        tokenized_data.append(temp_X)
    return tokenized_data

In [6]:
# song_id로 rank_df 행 찾기
def find_rank_song(song_id):
    return rank_df[rank_df['song_id'] ==  song_id]

# album_id로 rank_df 행 찾기
def find_rank_album(album_id):
    return rank_df[rank_df['album_id'] ==  album_id]

# album_id로 info_df 행 찾기
def find_info_id(album_id):
    return info_df[info_df['AlbumID'] == album_id]

# song_id를 넣으면 genre를 반환
def find_genre_song(song_id):
    album_id = find_rank_song(song_id)['album_id'].values[0]
    genre = find_info_id(album_id)['Genre'].values[0]
    return genre

# song_id_list를 넣으면 genre_list를 반환
def find_genres_songs(song_id_list):
    result = []
    for song_id in song_id_list:
        genre = find_genre_song(song_id)
        result.append(genre)
    return result

In [7]:
# unique한 장르 list 구하기
def make_unique_lst(df_column):
    unique_lst = df_column.unique()
    # print(len(unique_lst)) #89

    # 한 행의 장르가 '댄스, 랩/힙합, 록/메탈' 로 콤마로 여러개가 표시되어 있는 경우가 있음
    split_lst = [val.split(', ') for val in unique_lst]
    sum_lst = sum(split_lst, [])  # 리스트 내 리스트 합치기
    unique_lst = list(set(sum_lst))  # 중복 제거
    return unique_lst

In [8]:
# 리턴 -> dict {"A" :1, "B" :2, ... }  형식
def make_word_to_idx(unique_lst):
    unique_lst = make_unique_lst(unique_lst)
    word_to_idx = {val: idx for idx, val in enumerate(unique_lst)}
    return word_to_idx

In [9]:
# 2차원 리스트를 넣으면 단어 각각을 숫자와 매칭시키고 숫자로 바꾼다.

# input
# 숫자로 바꾸고 싶은 데이터 프레임 열

# return
# 단어를 숫자로 매핑한 2차원 배열, word_to_idx

# column_list: 데이터 프레임 열
def make_idx_list(column_lst):
    df_column_list = [tmp.split(", ") for tmp in column_lst]
    word_to_idx = make_word_to_idx(column_lst)

    new_column = []
    for column in df_column_list:
        tmp_lst = []
        for one_column in column:
            tmp_lst.append(word_to_idx[one_column])
        new_column.append(tmp_lst)

    return new_column, word_to_idx


### 전처리 - 차트파인더

#### 비어있는 행 확인

In [ ]:
info_df.isna().sum()

Unnamed: 0      0
AlbumID         0
Release_date    0
Genre           0
Publisher       0
Agency          0
dtype: int64

In [ ]:
song_df.isna().sum()

Song_ID    0
Lyrics     0
dtype: int64

In [ ]:
rank_df.isna().sum()

Unnamed: 0    0
year          0
rank          0
song_name     0
artist        0
likes         0
album_id      0
song_id       0
dtype: int64

#### song_df에 숫자로 매칭된 genre 열 생성

In [ ]:
song_df['genre'] = find_genres_songs(song_df['Song_ID'])

In [ ]:
song_df

,Song_ID,Lyrics,genre
0,34373632,어린 햇살 아래서뛰어놀곤 했었던가쁜 숨결굽이진 골목 지나길을 따라가보면같은 기억어른...,"국내드라마, 발라드"
1,8124417,꽃이 언제 피는지 그딴 게 뭐가 중요한데날씨가 언제 풀리는지 그딴 거 알면 뭐 할 ...,"인디음악, 포크/블루스"
2,2705410,늦게 다니지좀 마술은 멀리좀 해봐열살짜리 애처럼 말을 안듣니정말 웃음만 나와누가 누...,발라드
3,30205955,난 또 행복한 척 더 더 행복한 척난 또 행복한 척 더 더 행복한 척아무에게도 말하...,R&B/Soul
4,3590149,밥을 먹어도 혼자야구를 봐도 혼자거리를 걸어도 나 혼자혼자이기 싫은데또 방에 누워도...,발라드
...,...,...,...
1102,3713013,닫힌 내 가슴은 누구도 사랑할 수가 없다 그렇게 믿었는데 어느새 내 가슴이 열리...,댄스
1103,4020215,네가 떠나고 늘 슬프다 아무리 슬퍼해도 슬픔이 모자라 맘 놓고 아파할 곳을 찾아 숨...,랩/힙합
1104,2797558,오 내 맘이야 맘이야 맘이 맘이야 오 내 맘이야 맘이야 맘이 맘이야 Ma Ma Ma...,"댄스, 발라드"
1105,31873020,그대와 자주 가던 그 술집에혼자 널 생각하며 소주 한잔해그대가 좋아하던 김치찌개를가...,발라드


In [ ]:
song_df['genre'], genre_to_idx = make_idx_list(song_df['genre'])

In [ ]:
genre_to_idx

{'클래식': 0,
 '애시드/퓨전/팝': 1,
 '록/메탈': 2,
 'R&B/Soul': 3,
 '크로스오버': 4,
 '인디음악': 5,
 '포크/블루스': 6,
 '국내드라마': 7,
 '일렉트로니카': 8,
 '성인가요/트로트': 9,
 '댄스': 10,
 '랩/힙합': 11,
 '발라드': 12,
 '재즈': 13}

In [ ]:
song_df.tail(3)

,Song_ID,Lyrics,genre
1104,2797558,오 내 맘이야 맘이야 맘이 맘이야 오 내 맘이야 맘이야 맘이 맘이야 Ma Ma Ma...,"[10, 12]"
1105,31873020,그대와 자주 가던 그 술집에혼자 널 생각하며 소주 한잔해그대가 좋아하던 김치찌개를가...,[12]
1106,2705407,Push Push Baby Oh Push BabyPush Push baby Just...,"[10, 12]"


#### song_df에 가사(Lyrics) 전처리

In [ ]:
song_df['Lyrics'] = preprocess(song_df['Lyrics'])

In [ ]:
song_df.tail(3)

,Song_ID,Lyrics,genre
1104,2797558,오 내 맘이야 맘이야 맘이 맘이야 오 내 맘이야 맘이야 맘이 맘이야 ...,"[10, 12]"
1105,31873020,그대와 자주 가던 그 술집에혼자 널 생각하며 소주 한잔해그대가 좋아하던 김치찌개를가...,[12]
1106,2705407,맘을 받아줘 달콤한 말로 감은 두 눈을 떠 감추려 하지마이미 넌...,"[10, 12]"


In [ ]:
song_df

,Song_ID,Lyrics,genre
0,34373632,어린 햇살 아래서뛰어놀곤 했었던가쁜 숨결굽이진 골목 지나길을 따라가보면같은 기억어른...,"[7, 12]"
1,8124417,꽃이 언제 피는지 그딴 게 뭐가 중요한데날씨가 언제 풀리는지 그딴 거 알면 뭐 할 ...,"[5, 6]"
2,2705410,늦게 다니지좀 마술은 멀리좀 해봐열살짜리 애처럼 말을 안듣니정말 웃음만 나와누가 누...,[12]
3,30205955,난 또 행복한 척 더 더 행복한 척난 또 행복한 척 더 더 행복한 척아무에게도 말하...,[3]
4,3590149,밥을 먹어도 혼자야구를 봐도 혼자거리를 걸어도 나 혼자혼자이기 싫은데또 방에 누워도...,[12]
...,...,...,...
1102,3713013,닫힌 내 가슴은 누구도 사랑할 수가 없다 그렇게 믿었는데 어느새 내 가슴이 열리...,[10]
1103,4020215,네가 떠나고 늘 슬프다 아무리 슬퍼해도 슬픔이 모자라 맘 놓고 아파할 곳을 찾아 숨...,[11]
1104,2797558,오 내 맘이야 맘이야 맘이 맘이야 오 내 맘이야 맘이야 맘이 맘이야 ...,"[10, 12]"
1105,31873020,그대와 자주 가던 그 술집에혼자 널 생각하며 소주 한잔해그대가 좋아하던 김치찌개를가...,[12]


In [ ]:
# 반복되는 가사를 어떻게 한번만 등장하게끔 만들 수 있을까?

In [ ]:
# 한 노래의 장르가 여러개일 경우 어떻게 처리하지?
# 예측할 때 하나라도 맞추면 맞았다고 해야하는지?
# 다중 레이블 분류 찾아보기

### 전처리 - 장르 스테디셀러

In [11]:
steady_song_df

,Song_ID,Genre,Lyrics
0,5808131,"발라드, 인디음악",보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...
1,81926,댄스,다시 널 사랑할 수 없다면내 모든 아픔까지 가져가네게 익숙해져 버린 나였기에너를 사...
2,73737,록/메탈,해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...
3,4800528,"발라드, R&B/Soul, 국내드라마",한참동안 그래 울기만 하잖아오늘도 그리워 하잖아그동안 힘들었지날보며 위로하는 그말 ...
4,4259861,발라드,늦은밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한숨...
...,...,...,...
2628,5537780,"인디음악, 포크/블루스",잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...
2629,3112951,발라드,아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...
2630,90105,랩/힙합,This is the tihs that y'all ain't ready for Th...
2631,3112955,발라드,하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...


In [12]:
steady_song_df.isna().sum()

Song_ID    0
Genre      0
Lyrics     0
dtype: int64

In [13]:
steady_song_df['Genre'], genre_to_idx = make_idx_list(steady_song_df['Genre'])

In [14]:
steady_song_df

,Song_ID,Genre,Lyrics
0,5808131,"[12, 6]",보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...
1,81926,[11],다시 널 사랑할 수 없다면내 모든 아픔까지 가져가네게 익숙해져 버린 나였기에너를 사...
2,73737,[4],해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...
3,4800528,"[12, 10, 5]",한참동안 그래 울기만 하잖아오늘도 그리워 하잖아그동안 힘들었지날보며 위로하는 그말 ...
4,4259861,[12],늦은밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한숨...
...,...,...,...
2628,5537780,"[6, 3]",잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...
2629,3112951,[12],아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...
2630,90105,[9],This is the tihs that y'all ain't ready for Th...
2631,3112955,[12],하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...


In [15]:
genre_to_idx

{'일렉트로니카': 0,
 '성인가요/트로트': 1,
 '애니메이션/웹툰': 2,
 '포크/블루스': 3,
 '록/메탈': 4,
 '국내드라마': 5,
 '인디음악': 6,
 '만화': 7,
 '키즈': 8,
 '랩/힙합': 9,
 'R&B/Soul': 10,
 '댄스': 11,
 '발라드': 12,
 '국내영화': 13}

### 전처리 - 장르 스테디셀러 가사 요약

In [ ]:
steady_sum_song_df

,Song_ID,Genre,Lyrics,Summarized Lyrics
0,5808131,"발라드, 인디음악",보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...,보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...
1,81926,댄스,다시 널 사랑할 수 없다면내 모든 아픔까지 가져가네게 익숙해져 버린 나였기에너를 사...,이젠 혼자 남겨진 내가낯설고 어색하긴 하지만모두 끝나버린 사랑이었기에너를 떠나 보낼...
2,73737,록/메탈,해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...,해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...
3,4800528,"발라드, R&B/Soul, 국내드라마",한참동안 그래 울기만 하잖아오늘도 그리워 하잖아그동안 힘들었지날보며 위로하는 그말 ...,널 보고싶다고 수만번 불러보면 너도 나를한번쯤 생각하는지 사랑이 올까요 너도 나와 ...
4,4259861,발라드,늦은밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한숨...,늦은 밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한...
...,...,...,...,...
2628,5537780,"인디음악, 포크/블루스",잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...,하루가 지나어쩔수 없이 돌아가던 길너는 내게 다가와시간을 멈추고 싶어내맘은 달라 아...
2629,3112951,발라드,아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...,헤어지자 난 다시 갈래 여자 그냥 가마음이 다쳐서 숨도 못 쉬겠어심장에 번져서죽을만...
2630,90105,랩/힙합,This is the tihs that y'all ain't ready for Th...,지누션과 함께면 너도 새로 태어난 힙합 빠삐용 아무도 아무도 날 몰라줄 때 빽 있어...
2631,3112955,발라드,하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...,길에서 널 만날 수 있다면 그 날처럼 널 보내지 않을 거야 눈물이 멈춘대도너를 지울...


In [ ]:
steady_sum_song_df.isna().sum()

Song_ID              0
Genre                0
Lyrics               0
Summarized Lyrics    0
dtype: int64

In [ ]:
steady_sum_song_df['Genre'], genre_to_idx = make_idx_list(steady_sum_song_df['Genre'])

In [ ]:
steady_sum_song_df

,Song_ID,Genre,Lyrics,Summarized Lyrics
0,5808131,"[10, 3]",보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...,보내야 할 때 보낼 수 있을까쉽게 받아들일 수 있니나는 그게 잘 안돼이별이 그렇게 ...
1,81926,[1],다시 널 사랑할 수 없다면내 모든 아픔까지 가져가네게 익숙해져 버린 나였기에너를 사...,이젠 혼자 남겨진 내가낯설고 어색하긴 하지만모두 끝나버린 사랑이었기에너를 떠나 보낼...
2,73737,[12],해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...,해질무렵 날 끌고간 발걸음눈떠보니 잊은줄 알았던 곳에아직도 너의 대한 미움이 남아 ...
3,4800528,"[10, 8, 6]",한참동안 그래 울기만 하잖아오늘도 그리워 하잖아그동안 힘들었지날보며 위로하는 그말 ...,널 보고싶다고 수만번 불러보면 너도 나를한번쯤 생각하는지 사랑이 올까요 너도 나와 ...
4,4259861,[10],늦은밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한숨...,늦은 밤 들어와 힘에 겨워 지쳐도 보고싶다 달려와 안겨준 사람아파도 내가 걱정돼 한...
...,...,...,...,...
2628,5537780,"[3, 7]",잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...,하루가 지나어쩔수 없이 돌아가던 길너는 내게 다가와시간을 멈추고 싶어내맘은 달라 아...
2629,3112951,[10],아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...,헤어지자 난 다시 갈래 여자 그냥 가마음이 다쳐서 숨도 못 쉬겠어심장에 번져서죽을만...
2630,90105,[11],This is the tihs that y'all ain't ready for Th...,지누션과 함께면 너도 새로 태어난 힙합 빠삐용 아무도 아무도 날 몰라줄 때 빽 있어...
2631,3112955,[10],하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...,길에서 널 만날 수 있다면 그 날처럼 널 보내지 않을 거야 눈물이 멈춘대도너를 지울...


In [ ]:
genre_to_idx

{'일렉트로니카': 0,
 '댄스': 1,
 '국내영화': 2,
 '인디음악': 3,
 '성인가요/트로트': 4,
 '애니메이션/웹툰': 5,
 '국내드라마': 6,
 '포크/블루스': 7,
 'R&B/Soul': 8,
 '만화': 9,
 '발라드': 10,
 '랩/힙합': 11,
 '록/메탈': 12,
 '키즈': 13}

### 데이터셋 샘플

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("jeanlee/kmhas_korean_hate_speech")

In [ ]:
# 훈련 데이터, 검증 데이터, 테스트 데이터를 로드합니다.

train = load_dataset("jeanlee/kmhas_korean_hate_speech", split="train")
validation = load_dataset("jeanlee/kmhas_korean_hate_speech", split="validation")
test = load_dataset("jeanlee/kmhas_korean_hate_speech", split="test")

In [ ]:
train['label']

In [ ]:
train['text']

### 데이터 준비하기

#### 데이터셋 만들기 - 훈련 데이터, 테스트 데이터 나누기

In [16]:
class SongDataset(Dataset):
    def __init__(self, x_data, y_data):
        # self.data = pd.DataFrame(data, columns=['lyrics', 'genre'])
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]

        return x, y

In [17]:
# 사용할 데이터 : song_df
X = song_df['Lyrics']
Y = song_df['genre']

NameError: ignored

In [18]:
# 사용할 데이터 : steady_song_df
X = steady_song_df['Lyrics']
Y = steady_song_df['Genre']

In [ ]:
# 사용할 데이터 : steady_sum_song_df
X = steady_sum_song_df['Summarized Lyrics']
Y = steady_sum_song_df['Genre']

In [19]:
# 학습데이터 테스트 데이터 나누기 - 80% 20%
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size = 0.2, random_state = 42
)

train_data, train_label, test_data, test_label

In [20]:
train_data = X_train.reset_index(drop=True)
train_label = y_train.reset_index(drop=True)
test_data = X_test.reset_index(drop=True)
test_label = y_test.reset_index(drop=True)

In [21]:
# train_dataset = SongDataset(train_data)
# test_dataset = SongDataset(test_data)
train_dataset = SongDataset(train_data, train_label)
test_dataset = SongDataset(test_data, test_label)

In [22]:
train_dataset.__getitem__(3)

('one two three yeahIf you wanna prettyevery wanna pretty안된다는 맘은 no no no noIf you wanna prettyevery wanna pretty어디서나 당당하게 걷기나와 맞는 옷에 또 받쳐주는 말투센스있는 포즈 그냥 되지는 않죠생활 상식은 기본 시사 상식은 선택다 끊임없는 노력이죠Girl Pretty Girl Pretty Girl조금도 망설일 것 없죠 난Beautiful Girl Beautiful GirlBeautiful yeah yeah yeah yeahGirl Pretty Girl Pretty Girl그냥 되진 않는거죠 난Beautiful Girl Beautifulyeah yeah yeah yeahCome on Beautiful GirlIf you wanna prettyevery wanna pretty안된다는 맘은 no no no noIf you wanna prettyevery wanna pretty어디서나 당당하게 걷기Girl Pretty Girl Pretty Girl조금도 망설일 것 없죠 난Beautiful Girl Beautiful GirlBeautiful yeah yeah yeah yeahGirl Pretty Girl Pretty Girl그냥 되진 않는거죠 난Beautiful Girl Beautifulyeah yeah yeah yeahCome on Beautiful Girl마음은 예쁘게 표정은 산뜻하게하루를 시작하면서 잊지 말아야 하죠두 눈을 깜박이며 살짝 미소 지으면이젠 모든게 완벽하죠Girl Pretty Girl Pretty Girl누구라도 될 수 있죠 난Beautiful Girl Beautifulyeah yeah yeah yeahyeah yeah yeah yeahGirl Pretty Girl Pretty Girl그냥 되진 않는거죠 난Beautiful Girl Beautiful GirlIf you wanna prettyevery wanna pretty안된다는 맘은 no no no noIf

#### 데이터셋 전처리

In [23]:
train_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', train_data))
test_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', test_data))

In [24]:
enc = MultiLabelBinarizer()
enc.fit([list(range(0, 14))])

def multi_label(label):
    enc_label = enc.transform(label)
    float_arr = np.vstack(enc_label[:]).astype(float)
    update_label = float_arr.tolist()
    return update_label

In [25]:
genre_to_idx

{'일렉트로니카': 0,
 '성인가요/트로트': 1,
 '애니메이션/웹툰': 2,
 '포크/블루스': 3,
 '록/메탈': 4,
 '국내드라마': 5,
 '인디음악': 6,
 '만화': 7,
 '키즈': 8,
 '랩/힙합': 9,
 'R&B/Soul': 10,
 '댄스': 11,
 '발라드': 12,
 '국내영화': 13}

In [26]:
train_label

0       [12, 5]
1          [10]
2           [4]
3          [11]
4           [1]
         ...   
2101       [12]
2102     [6, 3]
2103        [1]
2104       [11]
2105    [12, 6]
Name: Genre, Length: 2106, dtype: object

In [27]:
train_labels = multi_label(train_label)
test_labels = multi_label(test_label)

In [28]:
train_labels[2105]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

In [29]:
print(len(train_labels), len(train_labels[0]))
print(len(test_labels), len(test_labels[0]))

2106 14
527 14


#### 장르별 몇개 있는지 확인

In [ ]:
all_labels = train_labels + test_labels
len(train_labels + test_labels)

2633

In [ ]:
all_labels_df = pd.DataFrame(all_labels)

In [ ]:
all_labels_df.sum()

0     481.0
1       1.0
2     298.0
3     334.0
4     320.0
5       2.0
6     274.0
7     472.0
8       8.0
9      93.0
10    451.0
11      1.0
12     10.0
13    341.0
dtype: float64

In [ ]:
genre_to_idx

{'댄스': 0,
 '키즈': 1,
 '인디음악': 2,
 'R&B/Soul': 3,
 '랩/힙합': 4,
 '애니메이션/웹툰': 5,
 '포크/블루스': 6,
 '발라드': 7,
 '일렉트로니카': 8,
 '국내드라마': 9,
 '록/메탈': 10,
 '만화': 11,
 '국내영화': 12,
 '성인가요/트로트': 13}

#### BERT 토크나이저를 이용한 전처리

In [30]:
# 한국어 BERT 중 하나인 'klue/bert-base'를 사용.
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [31]:
# train_data, train_label, test_data, test_label

In [33]:
# max([len(lyrics) for lyrics in song_df['Lyrics']])
max([len(lyrics) for lyrics in steady_song_df['Lyrics']])
# max([len(lyrics) for lyrics in steady_sum_song_df['Summarized Lyrics']])

2501

In [34]:
tokenizer.tokenize('가나다')

['가나', '##다']

In [35]:
MAX_LEN = 400

def data_to_tensor(sentences, labels):
    # 정수 인코딩
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
     # pad_sequence: 패딩, 주어진 최대 길이보다 짧으면 뒤에 0으로 채워준다
    input_ids = pad_sequences(list(input_ids), maxlen=MAX_LEN, dtype="long",
                              truncating = "post", padding="post")
    attention_masks = []

    for seq in input_ids:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)

    tensor_inputs = torch.tensor(input_ids)
    tensor_labels = torch.tensor(labels)
    tensor_masks = torch.tensor(attention_masks)

    return tensor_inputs, tensor_labels, tensor_masks

train_sentences, test_sentences, train_labels, test_labels

In [36]:
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels)


In [37]:
train_labels.shape

torch.Size([2106, 14])

In [38]:
test_labels.shape

torch.Size([527, 14])

In [39]:
batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [40]:
print('훈련 데이터의 크기:', len(train_labels))
print('테스트 데이터의 크기:', len(test_labels))

훈련 데이터의 크기: 2106
테스트 데이터의 크기: 527


### GPU 사용

In [41]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [42]:
# GPU가 잘 사용되는지 테스트할 때는 아래 코드를 사용해 보세요
# torch.rand(10).to("cuda:0")
# torch.rand(10).cuda()

### 모델 만들기

In [43]:
num_labels = 14

model = BertForSequenceClassification.from_pretrained("klue/bert-base",
        num_labels=num_labels, problem_type="multi_label_classification")

model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [44]:
# 옵티마이저 선택
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [45]:
# 에포크 설정
epochs = 15
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [46]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))  # hh:mm:ss

In [47]:
def multi_label_metrics(predictions, labels, threshold=0.5):

    # 모델의 예측에 대해서 시그모이드 함수값을 통과시킨다. (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    print('sigmoid:', sigmoid, ', probs:', probs)

    # 만약 threshold 값을 넘는 경우에는 1로 예측했다고 간주한다.
    # threshold 값은 일반적으로 로지스틱 회귀 방식에 의하여 0.5를 선택하는 것이 일반적이다.
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    y_true = labels

    # 사용 가능한 메트릭들을 사용한다.
    accuracy = accuracy_score(y_true, y_pred)
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
    f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')

    # 메트릭 결과에 대해서 리턴
    metrics = {'accuracy': accuracy,
               'f1_macro': f1_macro_average,
               'f1_micro': f1_micro_average,
               'f1_weighted': f1_weighted_average,
               'roc_auc': roc_auc}

    return metrics


GPU out of memory 에러 발생시 캐시 지우기

In [48]:
import torch
torch.cuda.empty_cache()

import gc

gc.collect()

torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      | 432917 KiB | 432917 KiB | 432917 KiB |      0 B   |\n|       from large pool | 432384 KiB | 432384 KiB | 432384 KiB |      0 B   |\n|       from small pool |    533 KiB |    533 KiB |    533 KiB |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         | 432917 KiB | 432917 KiB | 432917 KiB |      0 B   |\n|       from large pool | 432384 KiB | 432384 KiB |

In [49]:
# 랜덤 시드값.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()

# loss값을 저장해서 loss 값이 제일 적은 경우에 모델을 저장한다.
loss_list = []

for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    t0 = time.time()
    total_loss = 0

    model.train()

    for step, batch in tqdm(enumerate(train_dataloader)):
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # gradient clipping if it is over a threshold
        optimizer.step()
        scheduler.step()

        model.zero_grad()


        if len(loss_list) < 1:
            loss_list.append(loss)
            continue

        # loss값을 loss_list로 리스트로 저장해서
        # 리스트에 제일 적은 loss 값보다 작은 경우에 모델을 저장한다.
        if loss < min(loss_list):
            torch.save(model.state_dict(), f"BERT_multilabel_model_{epoch_i}.pt")


        loss_list.append(loss)

    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

======== Epoch 1 / 15 ========


132it [03:46,  1.72s/it]



  Average training loss: 0.2773
  Training epcoh took: 0:03:47
======== Epoch 2 / 15 ========


132it [03:09,  1.44s/it]



  Average training loss: 0.2058
  Training epcoh took: 0:03:10
======== Epoch 3 / 15 ========


...


  Average training loss: 0.0319
  Training epcoh took: 0:02:48
======== Epoch 14 / 15 ========


132it [02:53,  1.32s/it]



  Average training loss: 0.0301
  Training epcoh took: 0:02:54
======== Epoch 15 / 15 ========


132it [02:48,  1.28s/it]


  Average training loss: 0.0288
  Training epcoh took: 0:02:49


In [50]:
epoch_i

14

In [59]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 구글 드라이브 경로 설정
%cd /content/drive/MyDrive/텍스트 프로젝트
%pwd

Mounted at /content/drive
/content/drive/MyDrive/텍스트 프로젝트


'/content/drive/MyDrive/텍스트 프로젝트'

In [62]:
torch.save(model.state_dict(), f"model_bert/BERT_multilabel_model_{epoch_i}.pt")

In [60]:
epoch_i

14

In [52]:
# 예측하기

t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [], []

for step, batch in tqdm(enumerate(test_dataloader)):
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for b in logits:
        accum_logits.append(list(b))

    for b in label_ids:
        accum_label_ids.append(list(b))

accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)

33it [00:14,  2.36it/s]


In [53]:
# 저저번에 돌림 - 비교용 (가사.csv)
results = multi_label_metrics(accum_logits, accum_label_ids)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))
print("ROC-AUC: {0:.4f}".format(results['roc_auc']))

sigmoid: Sigmoid() , probs: tensor([[0.0062, 0.9555, 0.0058,  ..., 0.0133, 0.0158, 0.0088],
        [0.0085, 0.0154, 0.0094,  ..., 0.0099, 0.0398, 0.0147],
        [0.0055, 0.0094, 0.0043,  ..., 0.7011, 0.0306, 0.0057],
        ...,
        [0.0057, 0.0090, 0.0047,  ..., 0.4303, 0.0170, 0.0061],
        [0.0060, 0.0110, 0.0047,  ..., 0.1090, 0.0393, 0.0065],
        [0.0036, 0.0125, 0.0061,  ..., 0.0127, 0.0327, 0.0089]])
Accuracy: 0.4307
F1 (Macro) Score: 0.3023
F1 (Micro) Score: 0.5158
F1 (Weighted) Score: 0.5102
ROC-AUC: 0.7223


In [54]:
# 저번에 돌림 (가사_스테디셀러.csv)
results = multi_label_metrics(accum_logits, accum_label_ids)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))
print("ROC-AUC: {0:.4f}".format(results['roc_auc']))

sigmoid: Sigmoid() , probs: tensor([[0.0062, 0.9555, 0.0058,  ..., 0.0133, 0.0158, 0.0088],
        [0.0085, 0.0154, 0.0094,  ..., 0.0099, 0.0398, 0.0147],
        [0.0055, 0.0094, 0.0043,  ..., 0.7011, 0.0306, 0.0057],
        ...,
        [0.0057, 0.0090, 0.0047,  ..., 0.4303, 0.0170, 0.0061],
        [0.0060, 0.0110, 0.0047,  ..., 0.1090, 0.0393, 0.0065],
        [0.0036, 0.0125, 0.0061,  ..., 0.0127, 0.0327, 0.0089]])
Accuracy: 0.4307
F1 (Macro) Score: 0.3023
F1 (Micro) Score: 0.5158
F1 (Weighted) Score: 0.5102
ROC-AUC: 0.7223


In [55]:
# 이번에 돌릴 예정 (가사_스테디셀러_요약.xlsx)
results = multi_label_metrics(accum_logits, accum_label_ids)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))
print("ROC-AUC: {0:.4f}".format(results['roc_auc']))

sigmoid: Sigmoid() , probs: tensor([[0.0062, 0.9555, 0.0058,  ..., 0.0133, 0.0158, 0.0088],
        [0.0085, 0.0154, 0.0094,  ..., 0.0099, 0.0398, 0.0147],
        [0.0055, 0.0094, 0.0043,  ..., 0.7011, 0.0306, 0.0057],
        ...,
        [0.0057, 0.0090, 0.0047,  ..., 0.4303, 0.0170, 0.0061],
        [0.0060, 0.0110, 0.0047,  ..., 0.1090, 0.0393, 0.0065],
        [0.0036, 0.0125, 0.0061,  ..., 0.0127, 0.0327, 0.0089]])
Accuracy: 0.4307
F1 (Macro) Score: 0.3023
F1 (Micro) Score: 0.5158
F1 (Weighted) Score: 0.5102
ROC-AUC: 0.7223


In [55]:
torch.save(model.state_dict(), "BERT_multilabel_model.pt")

In [56]:
# 모델 저장
# torch.save(model.state_dict(), path+"BERT_multilabel_model.pt")

In [57]:
# 모델 로드
# model.load_state_dict(torch.load(path+"BERT_multilabel_model.pt"))

### 예측

In [ ]:
num_labels = 14

model = BertForSequenceClassification.from_pretrained("klue/bert-base",
        num_labels=num_labels, problem_type="multi_label_classification")

model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# 모델 로드
epoch = 14
model.load_state_dict(torch.load(f"BERT_multilabel_model_{epoch}.pt"))

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512,
                return_all_scores=True, function_to_apply='sigmoid')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [63]:
genre_to_idx

{'일렉트로니카': 0,
 '성인가요/트로트': 1,
 '애니메이션/웹툰': 2,
 '포크/블루스': 3,
 '록/메탈': 4,
 '국내드라마': 5,
 '인디음악': 6,
 '만화': 7,
 '키즈': 8,
 '랩/힙합': 9,
 'R&B/Soul': 10,
 '댄스': 11,
 '발라드': 12,
 '국내영화': 13}

In [64]:
# 텍스트 파일로 저장하기
file = open("genre_to_idx.txt", "w")
file.write(str(genre_to_idx))
file.close()

In [ ]:
for idx, val in enumerate(genre_to_idx):
    print(idx, val)

0 성인가요/트로트
1 애니메이션/웹툰
2 인디음악
3 국내드라마
4 만화
5 키즈
6 랩/힙합
7 일렉트로니카
8 록/메탈
9 R&B/Soul
10 포크/블루스
11 댄스
12 발라드
13 국내영화


In [ ]:
label_dict = {f"LABEL_{idx}" : val for idx, val  in enumerate(genre_to_idx)}

In [1]:
def prediction(text):
    result = pipe(text)
    return [label_dict[res['label']] for res in result[0] if res['score'] > 0.5]

In [2]:
lyrics = '''
Bring the fire
거침없이 Yeah
기나긴 밤 이겨냈지
선명한 너의 그 상처는 빛났어
Oh
주먹 쥐고 턱을 올려
그래 Ad Astra
아득한 미래도 내 손안에 잡아
Feeling like a rockstar
Lights on us
쏟아지는 갈채속의 Fire
We go
We go
We go hard
Hard like a criminal
Hard like the beat
Hard like ma flow
너도 원했니
밤이 녹을 만큼
뜨겁고 진한
새로운 히트
멈추지
못 했다지?
Hard like a criminal
Hard like the beat
Hard like ma flow
구미 당겼지
Yeah
멋 내 봐 맛을 봐
누가 누가 와
너를 놀래키나
We go hard
걸음은 킹콩
그 놈 참 물건
잠시 한 눈 좀 팔아도
다시 챔피언
왕관은 주인을 되찾아내
세상의 끝까지
Yeah we don’t stop
Oh
시간이 아까워 Tic Toc
서둘러
Sold out
목말랐던 만큼 원하고 원해 봐
Feeling like a rockstar
Lights on us
걸어 나와 네 순간을 마셔
We bring the fire
We go
We go
We go hard
Hard like a criminal
Hard like the beat
Hard like ma flow
너도 원했니
밤이 녹을 만큼
뜨겁고 진한
새로운 히트
멈추지
못 했다지?
Hard like a criminal
Hard like the beat
Hard like ma flow
구미 당겼지
Yeah
멋 내 봐 맛을 봐
누가 누가 와
너를 놀래키나
We go hard
같이 가볼까
되감을 순 없어
지금 이 순간만 기억해 줘
No looking back
넌 알잖아
미쳐 봐 날아 봐
우린 강해
We get you right
KO
We gon’ knock you out
거기 MC count down to...
Zero
Yesir
We go
We go
We go hard
Hard like a criminal
Hard like the beat
Hard like ma flow
너도 원했지
밤이 녹을 만큼
뜨겁고 진한
새로운 히트
멈추지
못 했다지?
Hard like a criminal
Hard like the beat
Hard like ma flow
구미 당겼지
Yeah
멋 내 봐 맛을 봐
누가 누가 와
너를 놀래키나
We go hard
'''

In [14]:
lyrics = preprocess([lyrics])

In [15]:
lyrics

['거침없이 기나긴 밤 이겨냈지선명한 너의 그 상처는 빛났어주먹 쥐고 턱을 올려그래  아득한 미래도 내 손안에 잡아     쏟아지는 갈채속의              너도 원했니밤이 녹을 만큼뜨겁고 진한새로운 히트멈추지못 했다지         구미 당겼지멋 내 봐 맛을 봐누가 누가 와너를 놀래키나  걸음은 킹콩그 놈 참 물건잠시 한 눈 좀 팔아도다시 챔피언왕관은 주인을 되찾아내세상의 끝까지   시간이 아까워  서둘러 목말랐던 만큼 원하고 원해 봐     걸어 나와 네 순간을 마셔                너도 원했니밤이 녹을 만큼뜨겁고 진한새로운 히트멈추지못 했다지         구미 당겼지멋 내 봐 맛을 봐누가 누가 와너를 놀래키나  같이 가볼까되감을 순 없어지금 이 순간만 기억해 줘  넌 알잖아미쳐 봐 날아 봐우린 강해       거기                 너도 원했지밤이 녹을 만큼뜨겁고 진한새로운 히트멈추지못 했다지         구미 당겼지멋 내 봐 맛을 봐누가 누가 와너를 놀래키나']

In [ ]:
result = pipe(lyrics)
print(result)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[{'label': 'LABEL_0', 'score': 0.029911257326602936}, {'label': 'LABEL_1', 'score': 0.022087812423706055}, {'label': 'LABEL_2', 'score': 0.048050861805677414}, {'label': 'LABEL_3', 'score': 0.0493047796189785}, {'label': 'LABEL_4', 'score': 0.012937212362885475}, {'label': 'LABEL_5', 'score': 0.013269860297441483}, {'label': 'LABEL_6', 'score': 0.011392117477953434}, {'label': 'LABEL_7', 'score': 0.011901864781975746}, {'label': 'LABEL_8', 'score': 0.9795778393745422}, {'label': 'LABEL_9', 'score': 0.010791727341711521}, {'label': 'LABEL_10', 'score': 0.0196143239736557}, {'label': 'LABEL_11', 'score': 0.03009822778403759}, {'label': 'LABEL_12', 'score': 0.024744965136051178}, {'label': 'LABEL_13', 'score': 0.01704501360654831}]]


In [ ]:
prediction(lyrics)

['록/메탈']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')